In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature, filters
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
#from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict
from skimage.morphology import *
from skimage.feature import *
from skimage.filters import *
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                           classes, transitions, times): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = morphology.binary_opening(thresh_img, disk(8))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =blob_doh(img, min_sigma =8, max_sigma = 17, threshold = 0.007, num_sigma= 15, overlap=0.5)
    
    ghighlight = gaussian(
        opening(
             (thresh_img^phighlight)
                           , disk(1)), sigma = 0.65)
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs = blob_doh(img, min_sigma =9, max_sigma = 16, threshold = 0.005, num_sigma= 15, overlap=0.2)
    gblobs[:, 2] = gblobs[:, 2] * np.sqrt(2)
    

    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
   
    glas_on_glas_dists =np.array([connect((gblobs[i][1], gblobs[i][0]),(g_bond, fg_bond), gblobs[i+1:]) for i in range(ngblobs-1)])
    num_gconnections, num_fgconnections = reducer(glas_on_glas_dists)


    plas_on_plas_dists = np.array([connect((pblobs[i][1], pblobs[i][0]),(p_bond, fp_bond), pblobs[i+1:]) for i in range(npblobs-1)])
    num_pconnections, num_fpconnections = reducer(plas_on_plas_dists)


    num_connections = sum([num_pconnections, num_gconnections])
    
    #----------------------filter------------------------- can use actual filter
    """test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    if test: # keep frames that have enough paricles, and are not transitions"""
    clas =sideify(num_pconnections,num_gconnections, num_fpconnections, num_fgconnections )
    #clas = side
    #sides[side].append(frame_no)
    classes[clas]+=1
    #print(clas)

    if clas == 'ucf':
        broken_count += 1
        class_num = 0
        #print('ucf-broken')
        #print("class_num: %s" %class_num)
        #print("broken_count: %s" %broken_count)
        pass
    else:
        if origin == 'ucf':
            origin = last_class
            #print("reset origin", origin)
        else:
            pass
            
        if clas == last_class:
            class_num += 1
            #print("class_num: %s" %class_num)
        else:
            class_num = 0
            #print("class_reset")
            #print("class_num: %s" %class_num)
            
        if class_num >= class_thresh: 
            if broken_count >= transition_threshold or clas != origin:
                transitions[origin + "->" + clas]+= 1#((last_whole, frame_no))
                
                between_time = frame_no - state_start                               
                times[origin + "->" + clas][0].append(between_time)
                
                stable_time = last_whole - state_start
                times[origin + "->" + clas][1].append(stable_time)
                
                unstable_time = frame_no - last_whole                      
                times[origin + "->" + clas][2].append(unstable_time)
                
                
                state_start = frame_no
                
                #print(origin + "->" + clas)
                origin = clas
                #print("origin: %s" %origin)
            broken_count = 0
            #print("broken_count: %s" %broken_count)
            last_whole = frame_no
        else:
            pass


        
        filtrate_len += 1 
        last_class = clas
            
            
    """else:
        print("broken")
        class_num = 0
        print("class_num: %s" %class_num)
        broken_count += 1
        print("broken_count: %s" %broken_count)"""

    return [broken_count, class_num, origin, last_class, last_whole,state_start, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify(num_pconnections, num_gconnections, num_fpconnections, num_fgconnections):
    if num_fpconnections + num_fgconnections < 2:
        return "ucf"
    elif num_pconnections == 1 and num_gconnections ==0:# and num_mconnections==4: #gtotallengths>48
        return "p"
    elif num_pconnections == 0 and num_gconnections ==1:# and num_mconnections == 4:
        return "g"
    elif num_pconnections == 1 and num_gconnections ==1:# and num_mconnections ==3:
        return "b"
    else:
        return "ucf"
    
def connect(cv, radii, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    num_f_connections = 0
    norm_radius = 15
    total_separations = 0
    normalized_dists = []
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        total_separations += vector_d
        r1,r2 = radii
        if vector_d <= 2*r1 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_connections += 1
        if vector_d <= 2*r2 and vector_d > .2*r1: # blob centers closer than diameter of one blob
            num_f_connections += 1
        else:
            pass
        if vector_d <= 4*r1: 
            normalized_dists.append(vector_d/(2*norm_radius))
    return (num_connections, num_f_connections)

def reducer(connections):
    if len(connections)== 0:
        return [0, 0]
    else:
        return(np.sum(connections, axis=0)    )


def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames =glob("./tchakamau/W*half/*_21*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 4
num_full_keys = 4
num_tran_keys = 3
trans_threshes = np.linspace(25, 65, 4)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[20,30,40]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
fp_bond = 15*1.75
fg_bond = 15*1.8
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            sides = {"p":[], "g":[], "b":[], "ucf":[]}
            classes = {"p":[], "g":[], "b":[], "ucf":[]}
            transitions = [("p->p",[]),("p->g",[]), ("p->b",[]),
                           ("g->p",[]),("g->g",[]), ("g->b",[]),
                          ("b->p",[]),("b->g",[]), ("b->b",[])]

            transitions = OrderedDict(transitions)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            for fr in frames:
                frame = fr[:, :, 2]
                num_total_frames += 1
                org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                             sides, classes, transitions)
                broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            trans_len = len([y for x in transitions.values() for y in x])
            skeys = list(sides.keys())
            ckeys = list(classes.keys())
            tkeys = list(transitions.keys())
            slcs = [len(sides[x]) for x in skeys]
            clcs = [len(classes[x]) for x in ckeys]
            trans = [len(transitions[x]) for x in tkeys] #num of each transition
            ucf = len(sides["ucf"])


            Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Top_bottoms, Full_classif, N_transitions, (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_15.5.npy',trans_runs)


In [4]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames =glob("./tchakamau/De*both/*.avi")#glob("./tchakamau/W*half/*_21*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 4
num_full_keys = 4
num_tran_keys = 3
num_lengths = 3 # ways to measure the time betwen transitions, as it were
trans_threshes = np.linspace(40, 60, 2)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[22,35]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1
g_bond = 15*1
fp_bond = 15*1.4
fg_bond = 15*1.5
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        #Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            
            classes = [("p",0), ("g",0), ("b",0), ("ucf",0)]
            transitions = [("p->p",0),("p->g",0), ("p->b",0),
                           ("g->p",0),("g->g",0), ("g->b",0),
                          ("b->p",0),("b->g",0), ("b->b",0)]
            times = [("p->p",[[] for i in range(num_lengths)]),("p->g",[[] for i in range(num_lengths)]), ("p->b",[[] for i in range(num_lengths)]),
                           ("g->p",[[] for i in range(num_lengths)]),("g->g",[[] for i in range(num_lengths)]), ("g->b",[[] for i in range(num_lengths)]),
                          ("b->p",[[] for i in range(num_lengths)]),("b->g",[[] for i in range(num_lengths)]), ("b->b",[[] for i in range(num_lengths)])]

            transitions = OrderedDict(transitions)
            times = OrderedDict(times)            
            classes = OrderedDict(classes)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            state_start = 0
            frame_no = 0
            for fr in frames:
                frame = fr[:, :, 2]
                frame_no += 1
                num_total_frames += 1
                org = total_threshold_filter(frame, frame_no, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                                              classes, transitions, times)
                broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            #trans_len = len([y for x in transitions.values() for y in x])
            
            ckeys = list(classes.keys())
            clcs = [classes[x] for x in ckeys] # list of frame nums per class
            
            tkeys = list(times.keys())
            tims = [times[x] for x in tkeys]
            
            trkeys = list(transitions.keys())
            trans = [transitions[x] for x in trkeys] #num of each transition
            #ucf = len(sides["ucf"])

            #Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = clcs
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))            
            T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            print(T_transitions[vidnum])
            print(Full_classif[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Full_classif, (N_transitions, T_transitions), (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_15.5_timesbetween.npy',trans_runs)


['./tchakamau\\Density_matchedboth\\45693_15Vpp_0_hasslipsitwontcatchandorbits.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_1.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_2.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_3.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_4.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_5_pushed.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_6.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_7.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_8_c.avi', './tchakamau\\Density_matchedboth\\45693_15Vpp_8_d.avi']
Number files:  10
Processing vid 0 : ./tchakamau\Density_matchedboth\45693_15Vpp_0_hasslipsitwontcatchandorbits.avi
T threshold: 40.0
C threshold: 22
[[33.  0. 13.]
 [ 0.  0.  0.]
 [14.  0. 14.]]
[[[list([1023, 773, 626, 1079, 915, 372, 374, 443, 819, 161, 231, 445, 460, 1590, 715, 441, 2694, 1009, 259, 391, 900, 289, 554, 819, 255, 448, 821, 547, 461, 81, 272, 482, 2782])
   list([])
   list([643, 1919, 2

T threshold: 40.0
C threshold: 22
[[ 1.  0.  7.]
 [ 0.  0.  2.]
 [ 7.  2. 17.]]
[[[list([999]) list([]) list([2156, 47, 1044, 888, 421, 233, 122])]
  [list([]) list([]) list([145, 153])]
  [list([340, 258, 472, 1175, 249, 1287, 235]) list([364, 1131])
   list([299, 94, 188, 1312, 705, 910, 1888, 206, 399, 367, 823, 1312, 1631, 531, 437, 355, 526])]]

 [[list([754]) list([]) list([1961, 4, 372, 832, 1, 58, 50])]
  [list([]) list([]) list([95, 3])]
  [list([11, 169, 429, 539, 4, 1056, 81]) list([281, 35])
   list([105, 6, 0, 934, 501, 784, 1731, 42, 67, 94, 10, 29, 988, 413, 1, 148, 2])]]

 [[list([245]) list([]) list([195, 43, 672, 56, 420, 175, 72])]
  [list([]) list([]) list([50, 150])]
  [list([329, 89, 43, 636, 245, 231, 154]) list([83, 1096])
   list([194, 88, 188, 378, 204, 126, 157, 164, 332, 273, 813, 1283, 643, 118, 436, 207, 524])]]]
[ 4626.  1949. 11526.  7576.]
Processing vid 8 : ./tchakamau\Density_matchedboth\45693_15Vpp_8_c.avi
T threshold: 40.0
C threshold: 22
[[1. 0. 1.

T threshold: 40.0
C threshold: 35
[[ 0.  0.  6.]
 [ 0.  8.  4.]
 [ 7.  3. 53.]]
[[[list([]) list([]) list([902, 311, 1575, 772, 1094, 791])]
  [list([]) list([726, 610, 742, 445, 422, 298, 1060, 2298])
   list([557, 1638, 696, 390])]
  [list([738, 460, 776, 94, 164, 525, 683]) list([728, 361, 338])
   list([183, 516, 326, 418, 351, 314, 1421, 683, 688, 3053, 1483, 261, 320, 350, 601, 568, 644, 414, 522, 496, 296, 485, 416, 545, 459, 506, 584, 504, 629, 314, 1050, 288, 481, 372, 364, 739, 1340, 665, 664, 401, 905, 518, 221, 796, 798, 340, 436, 1053, 3292, 350, 618, 306, 289])]]

 [[list([]) list([]) list([159, 247, 810, 90, 951, 611])]
  [list([]) list([0, 101, 87, 315, 41, 21, 788, 0]) list([1, 4, 176, 2])]
  [list([521, 157, 5, 22, 86, 275, 78]) list([390, 7, 134])
   list([42, 42, 138, 157, 142, 1, 1261, 549, 437, 319, 1193, 17, 18, 44, 67, 4, 280, 21, 14, 32, 4, 287, 131, 61, 104, 268, 229, 18, 459, 4, 460, 51, 279, 159, 140, 18, 351, 327, 342, 3, 425, 283, 0, 38, 110, 5, 5, 408, 27

T threshold: 60.0
C threshold: 22
[[ 0.  0.  4.]
 [ 0.  7.  2.]
 [ 4.  2. 11.]]
[[[list([]) list([]) list([69, 2464, 1872, 1890])]
  [list([]) list([543, 492, 578, 288, 317, 447, 369]) list([742, 271])]
  [list([2901, 1497, 2921, 985]) list([435, 670])
   list([2964, 624, 319, 465, 1117, 1220, 955, 1040, 878, 352, 530])]]

 [[list([]) list([]) list([6, 2030, 696, 1226])]
  [list([]) list([345, 2, 303, 125, 20, 159, 150]) list([6, 0])]
  [list([2788, 732, 2736, 774]) list([187, 102])
   list([2721, 300, 3, 79, 873, 770, 206, 550, 689, 45, 72])]]

 [[list([]) list([]) list([63, 434, 1176, 664])]
  [list([]) list([198, 490, 275, 163, 297, 288, 219]) list([736, 271])]
  [list([113, 765, 185, 211]) list([248, 568])
   list([243, 324, 316, 386, 244, 450, 749, 490, 189, 307, 458])]]]
[ 4235.  2653. 15661.  9754.]
Processing vid 5 : ./tchakamau\Density_matchedboth\45693_15Vpp_5_pushed.avi
T threshold: 60.0
C threshold: 22
[[ 2.  0.  9.]
 [ 0.  9.  3.]
 [ 9.  4. 31.]]
[[[list([3313, 732]) list(

T threshold: 60.0
C threshold: 35
[[6. 1. 5.]
 [0. 0. 1.]
 [5. 0. 2.]]
[[[list([1429, 362, 451, 563, 1179, 1052]) list([2871])
   list([827, 851, 717, 1086, 4121])]
  [list([]) list([]) list([589])]
  [list([960, 462, 1542, 1374, 1364]) list([]) list([1340, 1102])]]

 [[list([441, 144, 0, 313, 423, 663]) list([2547])
   list([397, 519, 5, 700, 4005])]
  [list([]) list([]) list([1])]
  [list([165, 347, 1431, 774, 768]) list([]) list([910, 110])]]

 [[list([988, 218, 451, 250, 756, 389]) list([324])
   list([430, 332, 712, 386, 116])]
  [list([]) list([]) list([588])]
  [list([795, 115, 111, 600, 596]) list([]) list([430, 992])]]]
[11758.   353.  6593.  6338.]
Processing vid 3 : ./tchakamau\Density_matchedboth\45693_15Vpp_3.avi
T threshold: 60.0
C threshold: 35
[[0. 0. 3.]
 [0. 0. 0.]
 [3. 0. 8.]]
[[[list([]) list([]) list([1027, 1562, 268])]
  [list([]) list([]) list([])]
  [list([693, 355, 267]) list([])
   list([1887, 616, 338, 499, 1239, 2366, 777, 1207])]]

 [[list([]) list([]) list

In [5]:
print(np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0]))

print(vidnum)

T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
#print(T_transitions)
T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
print(T_transitions, len(T_transitions), vidnum)

[[[list([479, 524, 270]) list([])
   list([68, 585, 603, 600, 690, 1933, 418, 1149, 1815, 271, 280, 1743])]
  [list([]) list([]) list([])]
  [list([628, 283, 310, 855, 764, 172, 398, 543, 579, 417, 1818])
   list([])
   list([810, 1266, 637, 1698, 595, 422, 1922, 1929, 4253, 1319, 521, 884, 350, 1485, 1039, 1033, 827, 1386])]]

 [[list([33, 9, 0]) list([])
   list([0, 318, 6, 13, 0, 1261, 166, 334, 1204, 175, 33, 1524])]
  [list([]) list([]) list([])]
  [list([13, 9, 173, 98, 4, 23, 135, 324, 260, 4, 1260]) list([])
   list([476, 933, 330, 968, 339, 141, 736, 1052, 3757, 307, 22, 313, 23, 21, 670, 155, 19, 1016])]]

 [[list([446, 515, 270]) list([])
   list([68, 267, 597, 587, 690, 672, 252, 815, 611, 96, 247, 219])]
  [list([]) list([]) list([])]
  [list([615, 274, 137, 757, 760, 149, 263, 219, 319, 413, 558])
   list([])
   list([334, 333, 307, 730, 256, 281, 1186, 877, 496, 1012, 499, 571, 327, 1464, 369, 878, 808, 370])]]]
9
[[[[[], [], []], [[], [], []], [[], [], []]], [[[], [], [